In [1]:
# Standard imports

import pandas as pd
import numpy as np

In [2]:
# Sklearn imports

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load data

In [3]:
features = ["Quiz", "HW", "Participation", "Midterm"]
target = "Final"
grades = pd.read_csv("../data/final/all_grades_no_outliers.csv")
X = grades[features]
y = grades[target]

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, shuffle=True, random_state=314
)

# Apply CV on models

In [4]:
# Define models

model_names = ["linear_regression", "ridge", "lasso", "knn", "random_forest", "gb_tree"]

models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    KNeighborsRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
]

parameters = [
    {},
    {"ridge__alpha": np.logspace(-3, 3, 7)},
    {"lasso__alpha": np.logspace(-3, 3, 7)},
    {"knn__n_neighbors": [1, 3, 5, 10, 20, 50]},
    {
        "random_forest__n_estimators": [1, 10, 100, 500],
        "random_forest__max_depth": [1, 2, 3, 4, 5],
    },
    {"gb_tree__n_estimators": [1, 10, 100, 500], "gb_tree__max_depth": [1, 2, 3, 4, 5]},
]

In [5]:
# Apply cross-validation for each model

grid_search = dict.fromkeys(model_names)

for model_name, model, params in zip(model_names, models, parameters):
    """
    Performs grid search for the given model and parameters, with standard scaling.
    """
    pipeline = Pipeline([("scaler", StandardScaler()), (model_name, model)])
    grid_search[model_name] = GridSearchCV(pipeline, param_grid=params, n_jobs=-1).fit(
        X_trainval, y_trainval
    )

In [6]:
# Check best score and parameters

results = {
    model: [grid_search[model].best_params_, grid_search[model].best_score_]
    for model in model_names
}

In [7]:
pd.DataFrame(results)

,linear_regression,ridge,lasso,knn,random_forest,gb_tree
0,{},{'ridge__alpha': 10.0},{'lasso__alpha': 0.1},{'knn__n_neighbors': 10},"{'random_forest__max_depth': 3, 'random_forest...","{'gb_tree__max_depth': 1, 'gb_tree__n_estimato..."
1,0.579475,0.579531,0.579731,0.517805,0.569572,0.579269


We see not much difference between linear regression, linear regression with regularization, and gradient boosted trees.

# With polynomial features

In [8]:
from sklearn.preprocessing import PolynomialFeatures

In [9]:
parameters_poly = [
    {**parameter, "poly_feat__degree": [1, 2, 3]} for parameter in parameters
]

In [10]:
# Apply cross-validation with polynomial featuresfor each model

grid_search_poly = dict.fromkeys(model_names)

for model_name, model, params in zip(model_names, models, parameters_poly):
    """
    Performs grid search for the given model and parameters, with standard scaling.
    """
    pipeline = Pipeline(
        [
            ("scaler", StandardScaler()),
            ("poly_feat", PolynomialFeatures()),
            (model_name, model),
        ]
    )
    grid_search_poly[model_name] = GridSearchCV(
        pipeline, param_grid=params, n_jobs=-1
    ).fit(X_trainval, y_trainval)

In [11]:
# Check best score and parameters

results_poly = {
    model: [grid_search_poly[model].best_params_, grid_search_poly[model].best_score_]
    for model in model_names
}

In [13]:
pd.DataFrame(results)

,linear_regression,ridge,lasso,knn,random_forest,gb_tree
0,{},{'ridge__alpha': 10.0},{'lasso__alpha': 0.1},{'knn__n_neighbors': 10},"{'random_forest__max_depth': 3, 'random_forest...","{'gb_tree__max_depth': 1, 'gb_tree__n_estimato..."
1,0.579475,0.579531,0.579731,0.517805,0.569572,0.579269


In [15]:
pd.DataFrame(results_poly)

,linear_regression,ridge,lasso,knn,random_forest,gb_tree
0,{'poly_feat__degree': 1},"{'poly_feat__degree': 1, 'ridge__alpha': 10.0}","{'lasso__alpha': 0.1, 'poly_feat__degree': 1}","{'knn__n_neighbors': 10, 'poly_feat__degree': 1}","{'poly_feat__degree': 1, 'random_forest__max_d...","{'gb_tree__max_depth': 1, 'gb_tree__n_estimato..."
1,0.579475,0.579531,0.579731,0.517805,0.575262,0.582297


As we see, polynomial features barely help.

# Feature importance

Let us quickly check feature importance for models that provides it. We do not use polynomial features since it barely improves the models, and choose the optimal parameters above.

In [17]:
gbtree = GradientBoostingRegressor(max_depth=1, n_estimators=100)

gbtree.fit(X_trainval, y_trainval)
print(f"Score on training set = {gbtree.score(X_trainval, y_trainval):.3f}")
print(f"Score on test set = {gbtree.score(X_test, y_test):.3f}")

Score on training set = 0.639
Score on test set = 0.625


In [18]:
for feature, weight in zip(X_trainval.columns, gbtree.feature_importances_):
    print(feature + " :", weight)

Quiz : 0.03009000710661984
HW : 0.046660080900109153
Participation : 0.0
Midterm : 0.923249911993271


In [19]:
forest = RandomForestRegressor(max_depth=3, n_estimators=500)

forest.fit(X_trainval, y_trainval)
print(f"Score on training set = {forest.score(X_trainval, y_trainval):.3f}")
print(f"Score on test set = {forest.score(X_test, y_test):.3f}")

Score on training set = 0.636
Score on test set = 0.629


In [20]:
for feature, weight in zip(X_trainval.columns, forest.feature_importances_):
    print(feature + " :", weight)

Quiz : 0.01798564241120142
HW : 0.0320681354058496
Participation : 0.001509637488409295
Midterm : 0.9484365846945397


Clearly, the midterm is what matters most to the final exam score. Considering that the midterm and the final are done in class and they other assignments at home, it makes sense!

# Statistical analysis

Let us perform a simple statstical analysis for the essentially optimal linear regression model.

### Weights

In [21]:
# Train model on the whole training test.

linreg = LinearRegression()

linreg.fit(X_trainval, y_trainval)
print(f"Score on training set = {linreg.score(X_trainval, y_trainval):.3f}")
print(f"Score on test set = {linreg.score(X_test, y_test):.3f}")

Score on training set = 0.588
Score on test set = 0.640


In [ ]:
# Check coefficients

print("Intercept:", linreg.intercept_)
for feature, weight in zip(X_trainval.columns, linreg.coef_):
    print(feature + " weight:", weight)

Intercept: 0.2796187218526427
Quiz weight: 0.12421784011649246
HW weight: 1.0411002929160798
Participation weight: 0.023804618072132827
Midterm weight: 0.7204030695516952


Essentially, this means that everyone starts with 28 "free" points.
- Participation grade is essentially irrelevant. Getting full marks (5) would only increase the final exam grade by 0.1, on average. *This does not mean that participation does not matter!*
- The same can be said for quizzes (full marks 20).
- The HW grade seem to have a massive effect: a student with 20/20 on the HW can expect a final exam grade 20*1.04 ~ 21 points higher. Nonetheless, this is essentially due to the fact that most students get very high HW grades. This can be explained by the fact that is an at-home assignment with no time limit, lax grading, or academic dishonesty. Essentially, the HW grade can almost be counted as part of the intercept, and only students who really slack get low HW grades.
- As expected, the midterm is what matters the most, and each point on the midterm translates to 0.72 points on the final on average. For instance, a student with a 90 on the midterm can expect 90*0.72 + 28 ~ 93 on the final.

### p-values

In [24]:
import statsmodels.api as sm

In [25]:
X_train_sm = sm.add_constant(X_trainval)

In [26]:
model = sm.OLS(y_trainval, X_train_sm).fit()

In [27]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Final   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     213.6
Date:                Wed, 25 Jun 2025   Prob (F-statistic):          1.00e-113
Time:                        20:47:07   Log-Likelihood:                -2405.4
No. Observations:                 603   AIC:                             4821.
Df Residuals:                     598   BIC:                             4843.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.2796      4.436      0.063

The p-values confirm the previous findings.

### Prediction

We predict the final exam grade of a few students, including confidence and prediction intervals. *Note that we need to include 1 for the intercept.*

We consider the following students.
- One who does excellently / poorly / average on all the assignments.
- One who does great on at-home assignments, poorly on the midterm exams.
- One who does poorly on at-home assignments, great on the midterm exams.
- One who does not participate or does honework, but does fine on quizzes and midtern.
- One who participates and does well on homework, but does average on quizzes and midterm.

In [32]:
X_pred = [[1, 18, 19, 5, 96],
          [1, 5, 12, 2, 50],
          [1, 12, 15, 5, 70], 
          [1, 18, 20, 5, 50],
          [1, 7, 2, 1, 85],
          [1, 18, 0, 0, 80],
          [1, 10, 18, 5, 60]
          ] 


In [ ]:
# Get 95% confidence interval for each student.

pred = model.get_prediction(X_pred)
pred.conf_int()

array([[89.92931659, 93.21900976],
       [44.4058365 , 54.5175118 ],
       [65.58108091, 70.2868694 ],
       [56.91268727, 62.04075727],
       [55.56960084, 73.40921859],
       [48.48867887, 71.80689195],
       [60.25133383, 66.95828549]])

In [42]:
# Get 95% prediction interval for each student.

pred.summary_frame()[['obs_ci_lower', 'obs_ci_upper']]

,obs_ci_lower,obs_ci_upper
0,65.752385,117.395942
1,23.201053,75.722295
2,42.057445,93.810506
3,33.580139,85.373305
4,37.219981,91.758838
5,31.863629,88.431942
6,37.618188,89.591432
